In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from data.DWSNets_dataset import DWSNetsDataset
import os

dir_path = os.path.dirname(os.path.abspath(os.getcwd()))
data_root = os.path.join(dir_path, "adl4cv", "datasets", "DWSNets", "mnist-inrs")

train_dataset = DWSNetsDataset(data_root)


In [7]:
train_dataset[0]

OrderedDict({'seq.0.weight': tensor([[-4.0755e-03,  3.0275e-02],
        [ 3.7985e-02,  3.5304e-02],
        [-6.5506e-02,  5.7733e-02],
        [ 1.7344e-02, -1.8718e-02],
        [ 3.0639e-02, -1.8703e-02],
        [ 5.9275e-02, -5.5776e-03],
        [-1.1590e-01,  1.9634e-02],
        [ 2.6851e-02, -2.4524e-02],
        [ 6.5062e-03,  1.1068e-01],
        [-9.7189e-02,  1.0473e-01],
        [ 6.1123e-02,  1.6021e-02],
        [-4.6886e-02, -3.9856e-02],
        [-7.6424e-02, -6.5537e-03],
        [ 3.6053e-02,  6.2836e-04],
        [ 5.1460e-02, -3.5410e-02],
        [ 1.5411e-02, -3.3180e-03],
        [ 2.2220e-01,  9.5588e-02],
        [-2.1557e-02,  2.9739e-02],
        [-1.1905e-02,  3.7337e-02],
        [-6.2158e-03, -9.7376e-02],
        [ 1.2494e-01,  5.1267e-02],
        [-4.9377e-02, -9.5556e-02],
        [-1.1631e-01, -1.1108e-01],
        [-1.6368e-02,  2.2905e-02],
        [ 1.8129e-02, -9.8052e-03],
        [-3.9306e-03,  2.7790e-02],
        [-1.2015e-02, -3.1052e-02],

(tensor([-0.0041,  0.0303,  0.0380,  ..., -0.1311,  0.1318, -0.1846]), 1)

In [15]:
from networks.INR import INR
from typing import List, Tuple, Union

import numpy as np
import torch

def make_coordinates(
    shape: Union[Tuple[int], List[int]],
    bs: int,
    coord_range: Union[Tuple[int], List[int]] = (-1, 1),
) -> torch.Tensor:
    x_coordinates = np.linspace(coord_range[0], coord_range[1], shape[0])
    y_coordinates = np.linspace(coord_range[0], coord_range[1], shape[1])
    x_coordinates, y_coordinates = np.meshgrid(x_coordinates, y_coordinates)
    x_coordinates = x_coordinates.flatten()
    y_coordinates = y_coordinates.flatten()
    coordinates = np.stack([x_coordinates, y_coordinates]).T
    coordinates = np.repeat(coordinates[np.newaxis, ...], bs, axis=0)
    return torch.from_numpy(coordinates).type(torch.float)


inr_kwargs={"n_layers": 3, "in_dim": 2, "up_scale": 16}
image_size=(28, 28)

inr = INR(**inr_kwargs)
print(train_dataset[0][0])
inr.load_state_dict(train_dataset[0][0])
inr.eval()
input = make_coordinates(image_size, 1)
with torch.no_grad():
    image = inr(input)
    image = image.view(*image_size, -1)
    image = image.permute(2, 0, 1)

print(image)



OrderedDict({'seq.0.weight': tensor([[-4.0755e-03,  3.0275e-02],
        [ 3.7985e-02,  3.5304e-02],
        [-6.5506e-02,  5.7733e-02],
        [ 1.7344e-02, -1.8718e-02],
        [ 3.0639e-02, -1.8703e-02],
        [ 5.9275e-02, -5.5776e-03],
        [-1.1590e-01,  1.9634e-02],
        [ 2.6851e-02, -2.4524e-02],
        [ 6.5062e-03,  1.1068e-01],
        [-9.7189e-02,  1.0473e-01],
        [ 6.1123e-02,  1.6021e-02],
        [-4.6886e-02, -3.9856e-02],
        [-7.6424e-02, -6.5537e-03],
        [ 3.6053e-02,  6.2836e-04],
        [ 5.1460e-02, -3.5410e-02],
        [ 1.5411e-02, -3.3180e-03],
        [ 2.2220e-01,  9.5588e-02],
        [-2.1557e-02,  2.9739e-02],
        [-1.1905e-02,  3.7337e-02],
        [-6.2158e-03, -9.7376e-02],
        [ 1.2494e-01,  5.1267e-02],
        [-4.9377e-02, -9.5556e-02],
        [-1.1631e-01, -1.1108e-01],
        [-1.6368e-02,  2.2905e-02],
        [ 1.8129e-02, -9.8052e-03],
        [-3.9306e-03,  2.7790e-02],
        [-1.2015e-02, -3.1052e-02],

TypeError: Expected state_dict to be dict-like, got <class 'torch.Tensor'>.

In [14]:
train_dataset[0][0]

OrderedDict({'seq.0.weight': tensor([[-4.0755e-03,  3.0275e-02],
        [ 3.7985e-02,  3.5304e-02],
        [-6.5506e-02,  5.7733e-02],
        [ 1.7344e-02, -1.8718e-02],
        [ 3.0639e-02, -1.8703e-02],
        [ 5.9275e-02, -5.5776e-03],
        [-1.1590e-01,  1.9634e-02],
        [ 2.6851e-02, -2.4524e-02],
        [ 6.5062e-03,  1.1068e-01],
        [-9.7189e-02,  1.0473e-01],
        [ 6.1123e-02,  1.6021e-02],
        [-4.6886e-02, -3.9856e-02],
        [-7.6424e-02, -6.5537e-03],
        [ 3.6053e-02,  6.2836e-04],
        [ 5.1460e-02, -3.5410e-02],
        [ 1.5411e-02, -3.3180e-03],
        [ 2.2220e-01,  9.5588e-02],
        [-2.1557e-02,  2.9739e-02],
        [-1.1905e-02,  3.7337e-02],
        [-6.2158e-03, -9.7376e-02],
        [ 1.2494e-01,  5.1267e-02],
        [-4.9377e-02, -9.5556e-02],
        [-1.1631e-01, -1.1108e-01],
        [-1.6368e-02,  2.2905e-02],
        [ 1.8129e-02, -9.8052e-03],
        [-3.9306e-03,  2.7790e-02],
        [-1.2015e-02, -3.1052e-02],

tensor([-0.0041,  0.0303,  0.0380,  ..., -0.1311,  0.1318, -0.1846])

In [ ]:
def get_batch(split: str):
    # get batch from dataloader
    pass